In [45]:
import pandas as pd

In [47]:
df = pd.read_csv("h1b_disclosure_data_2015_2019.csv")

In [51]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [48]:
df.head()

,CASE_NUMBER,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,SOC_CODE,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,WORKSITE_CITY,WORKSITE_STATE_ABB,YEAR,WORKSITE_STATE_FULL,WORKSITE
0,I-200-09285-514415,CERTIFIED,IT CATS LLC,COMPUTER SYSTEMS ANALYSTS,15-1121,SENIOR BUSINESS ANALYST,Y,109116.8,"NORTH WALES,",PA,2015,PENNSYLVANIA,"NORTH WALES, PENNSYLVANIA"
1,I-200-09341-564186,CERTIFIED-WITHDRAWN,"XPERT TECHNOLOGIES, INC.","COMPUTER OCCUPATIONS, ALL OTHER*",15-1799,QUALITY ASSURANCE ANALYST,Y,58282.0,CAMP HILL,PA,2015,PENNSYLVANIA,"CAMP HILL, PENNSYLVANIA"
2,I-200-09342-274341,CERTIFIED,"ISR INFO WAY, INC.","SOFTWARE DEVELOPERS, APPLICATIONS",15-1132,SR. SOFTWARE DEVELOPER,Y,84906.0,CHICAGO,IL,2015,ILLINOIS,"CHICAGO, ILLINOIS"
3,I-200-09364-425868,CERTIFIED,MAGIC CIRCLE FILMS INTERNATIONAL LLC,FILM AND VIDEO EDITORS,27-4032,VIDEO EDITOR,Y,39333.0,AUBURN,NY,2015,NEW YORK,"AUBURN, NEW YORK"
4,I-200-10006-413683,WITHDRAWN,SAPHIRE SOLUTIONS INC,NETWORK AND COMPUTER SYSTEMS ADMINISTRATORS,15-1142,SYSTEMS ADMINISTRATOR,Y,68638.0,BETHPAGE,NY,2015,NEW YORK,"BETHPAGE, NEW YORK"


In [53]:
df.dtypes

CASE_NUMBER             object
CASE_STATUS             object
EMPLOYER_NAME           object
SOC_NAME                object
SOC_CODE                object
JOB_TITLE               object
FULL_TIME_POSITION      object
PREVAILING_WAGE        float64
WORKSITE_CITY           object
WORKSITE_STATE_ABB      object
YEAR                     int64
WORKSITE_STATE_FULL     object
WORKSITE                object
dtype: object

In [15]:
df.duplicated().sum()

5

In [17]:
df.drop_duplicates()

In [19]:
columns_needed = [
    "CASE_STATUS",
    "EMPLOYER_NAME",
    "SOC_NAME",
    "JOB_TITLE",
    "PREVAILING_WAGE",
    "WORKSITE_CITY",
    "WORKSITE_STATE_FULL",
    "YEAR"
]

df = df[columns_needed]

In [21]:
df.isnull().sum()

CASE_STATUS             0
EMPLOYER_NAME          21
SOC_NAME                6
JOB_TITLE               5
PREVAILING_WAGE         4
WORKSITE_CITY          28
WORKSITE_STATE_FULL     0
YEAR                    0
dtype: int64

In [55]:
df = df.dropna(subset=["PREVAILING_WAGE"])

In [57]:
df["EMPLOYER_NAME"] = df["EMPLOYER_NAME"].fillna("Unknown")
df["JOB_TITLE"] = df["JOB_TITLE"].fillna("Unknown")
df["SOC_NAME"] = df["SOC_NAME"].fillna("Unknown")
df["WORKSITE_CITY"] = df["WORKSITE_CITY"].fillna("Unknown")

In [59]:
def clean_status(x):
    if x in ["CERTIFIED", "CERTIFIED-WITHDRAWN"]:
        return "Approved"
    elif x == "DENIED":
        return "Denied"
    elif x == "WITHDRAWN":
        return "Withdrawn"
    else:
        return "Other"

df["CASE_STATUS_CLEAN"] = df["CASE_STATUS"].apply(clean_status)

In [61]:
df["APPROVAL_FLAG"] = df["CASE_STATUS_CLEAN"].apply(lambda x: 1 if x=="Approved" else 0)

In [63]:
df = df[df["PREVAILING_WAGE"] > 0]

In [65]:
df["PREVAILING_WAGE"].describe()

count    8.572010e+05
mean     9.540197e+04
std      1.755501e+06
min      4.500000e+03
25%      5.851000e+04
50%      7.032500e+04
75%      8.636200e+04
max      2.977853e+08
Name: PREVAILING_WAGE, dtype: float64

In [67]:
Q1 = df["PREVAILING_WAGE"].quantile(0.25)
Q3 = df["PREVAILING_WAGE"].quantile(0.75)
IQR = Q3 - Q1

upper_limit = Q3 + 1.5 * IQR

df = df[df["PREVAILING_WAGE"] <= upper_limit]

In [69]:
df["EMPLOYER_NAME"] = df["EMPLOYER_NAME"].str.strip().str.upper()
df["JOB_TITLE"] = df["JOB_TITLE"].str.strip().str.upper()
df["WORKSITE_CITY"] = df["WORKSITE_CITY"].str.strip().str.upper()
df["WORKSITE_STATE_FULL"] = df["WORKSITE_STATE_FULL"].str.strip().str.upper()

In [41]:
print(df.shape)
df.info()

(821744, 10)
<class 'pandas.core.frame.DataFrame'>
Index: 821744 entries, 0 to 857215
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   CASE_STATUS          821744 non-null  object 
 1   EMPLOYER_NAME        821744 non-null  object 
 2   SOC_NAME             821744 non-null  object 
 3   JOB_TITLE            821744 non-null  object 
 4   PREVAILING_WAGE      821744 non-null  float64
 5   WORKSITE_CITY        821744 non-null  object 
 6   WORKSITE_STATE_FULL  821744 non-null  object 
 7   YEAR                 821744 non-null  int64  
 8   CASE_STATUS_CLEAN    821744 non-null  object 
 9   APPROVAL_FLAG        821744 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 69.0+ MB


In [43]:
df.to_csv("h1b_cleaned.csv", index=False)

In [71]:
df.sample(10000).to_csv("h1b_sample.csv", index=False)